<a href="https://colab.research.google.com/github/arminsoltan/mohaymen/blob/main/Phase07-Name-Entity-Recognition/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install parsivar

     |████████████████████████████████| 36.2 MB 66 kB/s 
     |████████████████████████████████| 1.5 MB 35.3 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492971 sha256=0dcb3cfb47801e04a1a631bddb034576edee77b7dc51087bdb33c323ca857056
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=7ada0336885ef0ee6a669e34d2eb97c3273a97a3cd5e98adcd3db29c04aaaa44
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
import nltk

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
ex = 'my name is Ali and I live in Iran'
entities = nltk.ne_chunk(pos_tag(word_tokenize(ex)))
print(entities)

(S
  my/PRP$
  name/NN
  is/VBZ
  (PERSON Ali/NNP)
  and/CC
  I/PRP
  live/VBP
  in/IN
  (GPE Iran/NNP))


In [7]:
ex = 'من علی هستم و در ایران زندگی می کنم'
entities = nltk.ne_chunk(pos_tag(word_tokenize(ex)))
print(entities)

(S
  من/JJ
  (ORGANIZATION علی/NNP)
  هستم/NNP
  و/NNP
  در/NNP
  ایران/NNP
  زندگی/NNP
  می/NNP
  کنم/NN)


# NER using Look Up Tables

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Mohaymen/Phase07/country_list.csv', header=None)
df["en"] = pd.DataFrame([word.split()[1][1:-1] for word in df[0]])
df["fa"] = pd.DataFrame([' '.join(word.split()[2:])[1:-1] for word in df[0]])
df.drop([0], axis=1, inplace=True)
df.head()

,en,fa
0,AFG,افغانستان
1,ALA,جزایر آلند
2,ALB,آلبانی
3,DZA,الجزایر
4,ASM,ساموای آمریکا


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df.to_pickle('/content/drive/MyDrive/Mohaymen/Phase07/countries')

In [11]:
country = pd.read_pickle('/content/drive/MyDrive/Mohaymen/Phase07/countries')

Create Lookup function

In [12]:
def lookup(sentence):
  result = []
  for name in country.fa.values:
    if name in sentence:
      result += [('GRE', name)]
  return result

In [13]:
print(lookup("ایرانیان از دیرباز تا کنون حس خاصی به وطن خود داشته اند و همیشه وطن پرست هستند. در این مطلب هم زیباترین جمله های احساسی وطن دوستانه را آماد کرده ایم."))
print(lookup("الیزابت دوم ملکهٔ بریتانیا، ملکهٔ کشور کانادا نیز هست و قانون به امضاء و به نام او رسمیت و ضمانت اجرایی پیدا می‌کند."))

[('GRE', 'ایران')]
[('GRE', 'کانادا'), ('GRE', 'بریتانیا')]


# NER with Regex

## regex function

In [14]:
import re
def regex(sentence):
  result = []
  iter = re.finditer(r"\bکشور\b", sentence)
  indices = [m.end(0) for m in iter]

  for i in indices:
    result.append(('GRE', sentence[i+1:sentence[i+1:].find(" ")+(i+1)]))
  return result

In [15]:
sentence = 'من در کشور ایران زندگی میکنم. من کشور خود را دوست دارم. من کشورم ایران را دوست دارم. او برای سفر به کشور جمهوری چک می رود.'

In [16]:
print(regex(sentence))

[('GRE', 'ایران'), ('GRE', 'خود'), ('GRE', 'جمهوری')]


# NER with lookup table and Regex

In [17]:
def lookup_regex(sentence):
  result = []
  result += lookup(sentence)
  result += regex(sentence)
  return list(set(result))

In [18]:
sentence = 'من در ایران زندگی میکنم. او در کشور جمهوری چک زندگی می کند.'
print(lookup_regex(sentence))

[('GRE', 'ایران'), ('GRE', 'جمهوری چک'), ('GRE', 'جمهوری')]


# NER with Stanford NER and NLTK

In [19]:
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
install_java()

In [20]:
import zipfile
zip_path = '/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0.zip'
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall('/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0')
zip_ref.close()

In [22]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

jar = '/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/stanford-ner-4.2.0.jar'
model = '/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'

sentence = u"Twenty miles east of Reno, Nev., " \
    "where packs of wild mustangs roam free through " \
    "the parched landscape, Tesla Gigafactory 1 " \
    "sprawls near Interstate 80."

ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

words = nltk.word_tokenize(sentence)

print(ner_tagger.tag(words))

[('Twenty', 'O'), ('miles', 'O'), ('east', 'O'), ('of', 'O'), ('Reno', 'LOCATION'), (',', 'O'), ('Nev.', 'LOCATION'), (',', 'O'), ('where', 'O'), ('packs', 'O'), ('of', 'O'), ('wild', 'O'), ('mustangs', 'O'), ('roam', 'O'), ('free', 'O'), ('through', 'O'), ('the', 'O'), ('parched', 'O'), ('landscape', 'O'), (',', 'O'), ('Tesla', 'ORGANIZATION'), ('Gigafactory', 'ORGANIZATION'), ('1', 'O'), ('sprawls', 'O'), ('near', 'O'), ('Interstate', 'LOCATION'), ('80', 'LOCATION'), ('.', 'O')]


In [38]:
!unzip /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/stanford-ner.jar -d /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/

Archive:  /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/stanford-ner.jar
   creating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/META-INF/
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/META-INF/MANIFEST.MF  
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/edu/stanford/nlp/ie/crf/CRFBiasedClassifier.class  
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/edu/stanford/nlp/ie/crf/CRFClassifier.class  
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/edu/stanford/nlp/ie/crf/NERGUI.class  
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/edu/stanford/nlp/ie/ClassifierCombiner.class  
  inflating: /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/e

In [58]:
# import shutil
# shutil.rmtree('/content/ner')

In [65]:
%cd "/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17"
!java -cp "lib:*" -mx16g "edu.stanford.nlp.ie.crf.CRFClassifier" -prop "/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/prop.txt"


/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17
Invoked on Mon Oct 25 08:45:28 UTC 2021 with arguments: -prop /content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/prop.txt
useTypeSeqs2=true
noMidNGrams=true
trainFile=farsi_train_corpus.tsv
maxNGramLeng=6
maxLeft=1
saveFeatureIndexToDisk=true
useObservedSequencesOnly=true
serializeTo=dummy-ner-model-farsi.ser.gz
wordShape=chris2useLC
useClassFeature=true
useNGrams=true
useNext=true
usePrev=true
useTypeySequences=true
featureDiffThresh=0.05
usePrevSequences=true
qnSize=2
useTypeSeqs=true
useSequences=true
map=word=0,answer=1
useWord=true
numFeatures = 1196025
Time to convert docs to feature indices: 15.9 seconds
Current memory used: 745m
numClasses: 13 [0=O,1=B-pers,2=B-event,3=I-event,4=I-pers,5=B-loc,6=I-loc,7=B-pro,8=I-pro,9=B-fac,10=I-fac,11=B-org,12=I-org]
numDocuments: 10241
numDatums: 334734
numFeatures: 1196025
Time to convert docs to data/labels: 9.1 seconds
Curren

In [66]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

sentence = "اسم من آرمین هست و در جمهوری اسلامی ایران زندگی می کنم"

jar = "/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/stanford-ner.jar"
model = "/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17/dummy-ner-model-farsi.ser.gz"

ner_tagger = StanfordNERTagger(model, jar, encoding="utf8")

words = nltk.word_tokenize(sentence)
print(ner_tagger.tag(words))

[('اسم', 'O'), ('من', 'O'), ('آرمین', 'O'), ('هست', 'O'), ('و', 'O'), ('در', 'O'), ('جمهوری', 'B-org'), ('اسلامی', 'I-org'), ('ایران', 'I-org'), ('زندگی', 'O'), ('می', 'O'), ('کنم', 'O')]


In [67]:
%cd "/content/drive/MyDrive/Mohaymen/Phase07/stanford-ner-4.2.0/stanford-ner-2020-11-17"
!java -cp "stanford-ner-4.2.0.jar" edu.stanford.nlp.ie.crf.CRFClassifier -loadClassifier dummy-ner-model-farsi.ser.gz -testFile dev.txt

Streaming output truncated to the last 5000 lines.
خدمات	O	O
استاد	O	O
لطفی	B-pers	B-pers
در	O	O
معرفی	O	O
فلسفه	O	O
غرب	O	O
در	O	O
0	O	O
سال	O	O
اخیر	O	O
را	O	O
تمامی	O	O
اهل	O	O
فلسفه	O	O
و	O	O
فرهنگ	O	O
ارج	O	O
می‌نهند	O	O
و	O	O
لازم	O	O
است	O	O
که	O	O
حق‌شناسی	O	O
و	O	O
احترام	O	O
خود	O	O
را	O	O
به	O	O
این	O	O
انسان	O	O
شریف	O	O
که	O	O
در	O	O
گوشه	O	O
تنهائی	O	O
به	O	O
ترجمه	O	O
این‌همه	O	O
آثار	O	O
مهم	O	O
همت	O	O
گماشته	O	O
است	O	O
،	O	O
اظهار	O	O
کنیم	O	O
و	O	O
سپاسگزار	O	O
زحمات	O	O
او	O	O
باشیم	O	O
.	O	O

دوره	O	O
اول	O	O
را	O	O
ترجمه	O	O
آثار	O	O
افلاطون	B-pers	B-pers
و	O	O
فلوطین	B-pers	B-pers
بیان	O	O
کرد	O	O
و	O	O
ترجمه	O	O
آثار	O	O
یاسپرس	B-pers	B-pers
،	O	O
فیلسوف	O	O
آلمانی	O	O
را	O	O
دوره	O	O
دوم	O	O
کارهای	O	O
دکتر	O	O
لطفی	B-pers	B-pers
دانست	O	O
.	O	O

سپس	O	O
استاد	O	O
محمدحسن	B-pers	B-pers
لطفی	I-pers	I-pers
با	O	O
سپاسگزاری	O	O
از	O	O
بانیان	O	O
این	O	O
نشست	O	O
،	O	O
به	O	O
توجیه	O	O
این	O	O
نکته	O	O
پرداخت	O	O
که	O	O
چرا	O	O
سه	O	O
کتاب	B-pro	B-pro
مابعدالطبیعه